In [1]:
import pandas as pd
import h5py
import yaml
import numpy as np

In [ ]:
488 1 - 3
561 1 - 3
647 1.5-3.5

In [5]:
location = "\\\\5.5.5.250\\share01\\yipgroup\\Au_Aaron\\02164-FRIZZ_SM\\290524\\"
trials = ["A", "B", "C", "D", "E", "F", "G", "H"]
suffix = "_MMStack_Default.ome_locs_undrift"
suffix2 = "_MMStack_Default.ome_locs"
num = 5

frame = 1000
lp = 0.275
min_ph = 500
max_ph = 1750
min_sig = 1
max_sig = 2.9

In [8]:
for t in trials:
    for i in [0,1,2,3,4]:
        file = t + "_" + str(i+1)
        try:
            with h5py.File(location + file + "\\" + file + suffix + ".hdf5","r") as f:
                dat = f['locs'][...]
        except:
            print("ISSUE AT " + file)
            with h5py.File(location + file + "\\" + file + suffix2 + ".hdf5","r") as f:
                dat = f['locs'][...]
        data = np.rec.array(dat, dtype=dat.dtype)

        #Filter data 
        data = data[((data['lpx']**2 + data['lpy']**2) <= lp**2)
                   & (data['photons'] < max_ph) & (data['photons'] > min_ph)
                   & (data['sx']**2 + data['sy']**2 <= max_sig**2)
                    & (data['sx']**2 + data['sy']**2 >= min_sig**2)]

        #Save as new HDF5 file
        with h5py.File(location + file + ".hdf5",'w') as f:
            f.create_dataset("locs", data = data)
    
        #Ammend YAML file
        new_doc = {
            'Generated by': 'HDF combiner',
            'bg': None,
            'ellipticity': None,
            'frame': None,
            'lpx': [0, lp],
            'lpy': [0, lp],
            'net_gradient': None,
            'photons': [min_ph, max_ph],
            'sx': [min_sig, max_sig],
            'sy': [min_sig, max_sig],
            'x': None,
            'y': None    
        }

        with open(location+file+"\\"+ file + suffix + ".yaml", "r") as stream:
            docs = list(yaml.load_all(stream, Loader=yaml.UnsafeLoader))

        do = []
        for doc in docs:
            do.append(doc)    
        do.append(new_doc)

        with open(location + file + ".yaml", "w") as stream:
            yaml.dump_all(do, stream, default_flow_style=False)